In [1]:
# Import basic libraries.
import numpy as np
import pandas as pd

# For visualization
import matplotlib.pyplot as plt
import seaborn as sns
import re

import warnings
warnings.filterwarnings('ignore')

In [2]:
# read the book data set
books = pd.read_csv("Books.csv",encoding='latin-1') # In the data set having spaces so we us

In [3]:
ratings = pd.read_csv('Ratings.csv', encoding='latin-1')
users = pd.read_csv('Users.csv', encoding='latin-1')

In [4]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [5]:
users.rename(columns={'User-ID': 'userId'}, inplace=True)

In [6]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [7]:
ratings.rename(columns={'User-ID': 'userId', 'Book-Rating': 'bookRating'}, inplace=True)

In [8]:
print(books.shape)
print(ratings.shape)
print(users.shape)

(271360, 8)
(1149780, 3)
(278858, 3)


## Checking Books Data

In [9]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [10]:
books.duplicated().sum()

0

In [11]:
## Renaming columns
books.rename(columns={'Year-Of-Publication': 'yearOfPublication'}, inplace=True)
books.rename(columns={'Book-Title': 'bookTitle', 'Book-Author': 'bookAuthor','Publisher': 'publisher', 'Image-URL-S': 'imageUrlS', 'Image-URL-M': 'imageUrlM', 'Image-URL-L': 'imageUrlL'}, inplace=True)

In [12]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ISBN               271360 non-null  object
 1   bookTitle          271360 non-null  object
 2   bookAuthor         271359 non-null  object
 3   yearOfPublication  271360 non-null  object
 4   publisher          271358 non-null  object
 5   imageUrlS          271360 non-null  object
 6   imageUrlM          271360 non-null  object
 7   imageUrlL          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


- Here we can see Year-Of-Publication should have dtype as int, so now we check the unique values of Year-Of-Publication using unqiue() method, so that we can see is there any data because of which our data type is converted in object

In [13]:
books.yearOfPublication.unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984, 0, 1968, 1961, 1958, 1974,
       1976, 1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960,
       1966, 1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954,
       1950, 1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011,
       1925, 1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 2030,
       1911, 1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 2050, 1934,
       1910, 1933, 1902, 1924, 1921, 1900, 2038, 2026, 1944, 1917, 1901,
       2010, 1908, 1906, 1935, 1806, 2021, '2000', '1995', '1999', '2004',
       '2003', '1990', '1994', '1986', '1989', '2002', '1981', '1993',
       '1983', '1982', '1976', '1991', '1977', '1998', '1992', '1996',
       '0', '1997', '2001', '1974', '1968', '1987', '1984', '1988',
       '1963', '1956', '1970', '1985', '1978', '1973', '1980'

- We can see that some author names are mixed in year of publication data example : 'DK Publishing Inc' and 'Gallimard'

- Now we will check the row which have 'DK Publishing Inc' as yearOfPublication

In [14]:
books.loc[books.yearOfPublication == 'DK Publishing Inc',:]

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlS,imageUrlM,imageUrlL
209538,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...,NaN
221678,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...,NaN


- As we can see there are two books with mixed data, hence we will correct it

In [15]:
books.loc[books.ISBN == '078946697X','imageUrlL'] = 'http://images.amazon.com/images/P/078946697X.01.LZZZZZZZ.jpg'
books.loc[books.ISBN == '078946697X','imageUrlM'] = 'http://images.amazon.com/images/P/078946697X.01.MZZZZZZZ.jpg'
books.loc[books.ISBN == '078946697X','imageUrlS'] = 'http://images.amazon.com/images/P/078946697X.01.THUMBZZZ.jpg'
books.loc[books.ISBN == '078946697X','publisher'] = 'DK Publishing Inc'
books.loc[books.ISBN == '078946697X','yearOfPublication'] = '2000'
books.loc[books.ISBN == '078946697X','bookAuthor'] = 'Michael Teitelbaum'
books.loc[books.ISBN == '078946697X','bookTitle'] = 'DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)'

In [16]:
books.loc[books.ISBN == '0789466953','imageUrlL'] = 'http://images.amazon.com/images/P/0789466953.01.LZZZZZZZ.jpg'
books.loc[books.ISBN == '0789466953','imageUrlM'] = 'http://images.amazon.com/images/P/0789466953.01.MZZZZZZZ.jpg'
books.loc[books.ISBN == '0789466953','imageUrlS'] = 'http://images.amazon.com/images/P/0789466953.01.THUMBZZZ.jpg'
books.loc[books.ISBN == '0789466953','publisher'] = 'DK Publishing Inc'
books.loc[books.ISBN == '0789466953','yearOfPublication'] = '2000'
books.loc[books.ISBN == '0789466953','bookAuthor'] = 'James Buckley'
books.loc[books.ISBN == '0789466953','bookTitle'] = "DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)"

- Now data with 'Gallimard'

In [17]:
books.loc[books.yearOfPublication == 'Gallimard',:]

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlS,imageUrlM,imageUrlL
220731,2070426769,"Peuple du ciel, suivi de 'Les Bergers\"";Jean-M...",2003,Gallimard,http://images.amazon.com/images/P/2070426769.0...,http://images.amazon.com/images/P/2070426769.0...,http://images.amazon.com/images/P/2070426769.0...,NaN


In [18]:
books.loc[books.ISBN == '2070426769','imageUrlL'] = 'http://images.amazon.com/images/P/2070426769.01.LZZZZZZZ.jpg'
books.loc[books.ISBN == '2070426769','imageUrlM'] = 'http://images.amazon.com/images/P/2070426769.01.MZZZZZZZ.jpg'
books.loc[books.ISBN == '2070426769','imageUrlS'] = 'http://images.amazon.com/images/P/2070426769.01.THUMBZZZ.jpg'
books.loc[books.ISBN == '2070426769','publisher'] = 'Gallimard'
books.loc[books.ISBN == '2070426769','yearOfPublication'] = '2003'
books.loc[books.ISBN == '2070426769','bookAuthor'] = 'Jean-Marie Gustave Le ClÃ?Â©zio'
books.loc[books.ISBN == '2070426769','bookTitle'] = "Peuple du ciel, suivi de 'Les Bergers"

In [19]:
## Verifying the change in YearOfPublication
books.loc[(books.ISBN == '2070426769') | (books.ISBN == '078946697X') | (books.ISBN == '0789466953'),: ]

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlS,imageUrlM,imageUrlL
209538,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",Michael Teitelbaum,2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...
220731,2070426769,"Peuple du ciel, suivi de 'Les Bergers",Jean-Marie Gustave Le ClÃ?Â©zio,2003,Gallimard,http://images.amazon.com/images/P/2070426769.0...,http://images.amazon.com/images/P/2070426769.0...,http://images.amazon.com/images/P/2070426769.0...
221678,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",James Buckley,2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...


- As we have seen that dtype of YearOfPublication is object so we will convert it into numeric dtype

In [20]:
books.yearOfPublication = pd.to_numeric(books.yearOfPublication, errors='coerce')

In [21]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ISBN               271360 non-null  object
 1   bookTitle          271360 non-null  object
 2   bookAuthor         271359 non-null  object
 3   yearOfPublication  271360 non-null  int64 
 4   publisher          271358 non-null  object
 5   imageUrlS          271360 non-null  object
 6   imageUrlM          271360 non-null  object
 7   imageUrlL          271360 non-null  object
dtypes: int64(1), object(7)
memory usage: 16.6+ MB


In [22]:
books.yearOfPublication.unique()

array([2002, 2001, 1991, 1999, 2000, 1993, 1996, 1988, 2004, 1998, 1994,
       2003, 1997, 1983, 1979, 1995, 1982, 1985, 1992, 1986, 1978, 1980,
       1952, 1987, 1990, 1981, 1989, 1984,    0, 1968, 1961, 1958, 1974,
       1976, 1971, 1977, 1975, 1965, 1941, 1970, 1962, 1973, 1972, 1960,
       1966, 1920, 1956, 1959, 1953, 1951, 1942, 1963, 1964, 1969, 1954,
       1950, 1967, 2005, 1957, 1940, 1937, 1955, 1946, 1936, 1930, 2011,
       1925, 1948, 1943, 1947, 1945, 1923, 2020, 1939, 1926, 1938, 2030,
       1911, 1904, 1949, 1932, 1928, 1929, 1927, 1931, 1914, 2050, 1934,
       1910, 1933, 1902, 1924, 1921, 1900, 2038, 2026, 1944, 1917, 1901,
       2010, 1908, 1906, 1935, 1806, 2021, 2012, 2006, 1909, 2008, 1378,
       1919, 1922, 1897, 2024, 1376, 2037], dtype=int64)

In [23]:
## Checking if any year has 0 value
books.loc[(books.yearOfPublication ==0), : ]

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlS,imageUrlM,imageUrlL
176,3150000335,Kabale Und Liebe,Schiller,0,"Philipp Reclam, Jun Verlag GmbH",http://images.amazon.com/images/P/3150000335.0...,http://images.amazon.com/images/P/3150000335.0...,http://images.amazon.com/images/P/3150000335.0...
188,342311360X,Die Liebe in Den Zelten,Gabriel Garcia Marquez,0,Deutscher Taschenbuch Verlag (DTV),http://images.amazon.com/images/P/342311360X.0...,http://images.amazon.com/images/P/342311360X.0...,http://images.amazon.com/images/P/342311360X.0...
288,0571197639,Poisonwood Bible Edition Uk,Barbara Kingsolver,0,Faber Faber Inc,http://images.amazon.com/images/P/0571197639.0...,http://images.amazon.com/images/P/0571197639.0...,http://images.amazon.com/images/P/0571197639.0...
351,3596214629,"Herr Der Fliegen (Fiction, Poetry and Drama)",Golding,0,Fischer Taschenbuch Verlag GmbH,http://images.amazon.com/images/P/3596214629.0...,http://images.amazon.com/images/P/3596214629.0...,http://images.amazon.com/images/P/3596214629.0...
542,8845229041,Biblioteca Universale Rizzoli: Sulla Sponda De...,P Coelho,0,Fabbri - RCS Libri,http://images.amazon.com/images/P/8845229041.0...,http://images.amazon.com/images/P/8845229041.0...,http://images.amazon.com/images/P/8845229041.0...
...,...,...,...,...,...,...,...,...
270794,014029953X,Foe (Essential.penguin S.),J.M. Coetzee,0,Penguin Books Ltd,http://images.amazon.com/images/P/014029953X.0...,http://images.amazon.com/images/P/014029953X.0...,http://images.amazon.com/images/P/014029953X.0...
270913,0340571187,Postmens House,Maggie Hemingway,0,Trafalgar Square,http://images.amazon.com/images/P/0340571187.0...,http://images.amazon.com/images/P/0340571187.0...,http://images.amazon.com/images/P/0340571187.0...
271094,8427201079,El Misterio De Sittaford,Agatha Christie,0,Editorial Molino,http://images.amazon.com/images/P/8427201079.0...,http://images.amazon.com/images/P/8427201079.0...,http://images.amazon.com/images/P/8427201079.0...
271182,0887781721,Tom Penny,Tony German,0,P. Martin Associates,http://images.amazon.com/images/P/0887781721.0...,http://images.amazon.com/images/P/0887781721.0...,http://images.amazon.com/images/P/0887781721.0...


- The value 0 for year is invalid, so setting invalid year as NaN

In [24]:
books.loc[(books.yearOfPublication == 0),'yearOfPublication'] = np.NAN

- There are some values in Year column which is greater than Year 2023 which is invalid, so setting invalid year as NaN

In [25]:
books.loc[(books["yearOfPublication"] >= 2023), : ]

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlS,imageUrlM,imageUrlL
37487,0671746103,MY TEACHER FRIED MY BRAINS (RACK SIZE) (MY TEA...,Coville,2030.0,Aladdin,http://images.amazon.com/images/P/0671746103.0...,http://images.amazon.com/images/P/0671746103.0...,http://images.amazon.com/images/P/0671746103.0...
55676,0671791990,MY TEACHER FLUNKED THE PLANET (RACK SIZE) (MY ...,Bruce Coville,2030.0,Aladdin,http://images.amazon.com/images/P/0671791990.0...,http://images.amazon.com/images/P/0671791990.0...,http://images.amazon.com/images/P/0671791990.0...
78168,0870449842,Crossing America,National Geographic Society,2030.0,National Geographic,http://images.amazon.com/images/P/0870449842.0...,http://images.amazon.com/images/P/0870449842.0...,http://images.amazon.com/images/P/0870449842.0...
80264,0140301690,Alice's Adventures in Wonderland and Through t...,Lewis Carroll,2050.0,Puffin Books,http://images.amazon.com/images/P/0140301690.0...,http://images.amazon.com/images/P/0140301690.0...,http://images.amazon.com/images/P/0140301690.0...
97826,0140201092,Outline of European Architecture (Pelican S.),Nikolaus Pevsner,2050.0,Penguin USA,http://images.amazon.com/images/P/0140201092.0...,http://images.amazon.com/images/P/0140201092.0...,http://images.amazon.com/images/P/0140201092.0...
116053,0394701658,Three Plays of Eugene Oneill,Eugene O'Neill,2038.0,Vintage Books USA,http://images.amazon.com/images/P/0394701658.0...,http://images.amazon.com/images/P/0394701658.0...,http://images.amazon.com/images/P/0394701658.0...
118294,3442436893,Das groÃ?Â?e BÃ?Â¶se- MÃ?Â¤dchen- Lesebuch.,Kathy Lette,2026.0,Goldmann,http://images.amazon.com/images/P/3442436893.0...,http://images.amazon.com/images/P/3442436893.0...,http://images.amazon.com/images/P/3442436893.0...
192993,0870446924,"Field Guide to the Birds of North America, 3rd...",National Geographic Society,2030.0,National Geographic,http://images.amazon.com/images/P/0870446924.0...,http://images.amazon.com/images/P/0870446924.0...,http://images.amazon.com/images/P/0870446924.0...
228173,0671266500,FOREST PEOPLE (Touchstone Books (Hardcover)),Colin M. Turnbull,2030.0,Simon &amp; Schuster,http://images.amazon.com/images/P/0671266500.0...,http://images.amazon.com/images/P/0671266500.0...,http://images.amazon.com/images/P/0671266500.0...
240169,0684718022,In Our Time: Stories (Scribner Classic),Ernest Hemingway,2030.0,Collier Books,http://images.amazon.com/images/P/0684718022.0...,http://images.amazon.com/images/P/0684718022.0...,http://images.amazon.com/images/P/0684718022.0...


In [26]:
books.loc[(books.yearOfPublication >= 2023),'yearOfPublication'] = np.NAN

In [27]:
books.yearOfPublication.fillna(round(books.yearOfPublication.mean()),inplace=True)
convert_dict = {'yearOfPublication': np.int64,}
books = books.astype(convert_dict)

- Here using .fillna(), we fill the NAN values with mean value of the year and convert the dtype to int

In [28]:
books_data = books[['ISBN', 'bookTitle','imageUrlS', 'imageUrlL']]
books = books[['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher', 'imageUrlM']]

- as of now we don't need img link so we save them in different dataframe

In [29]:
## Applying clean_text() function on bookTitle column
def clean_text(text):
    #removal of url
    text = re.sub(r'https?://\S+|www\.\S+|http?://\S+',' ',text) 
    
    #removal of html tags
    text = re.sub(r'<.*?>',' ',text) 
    
    text = re.sub("["
                           u"\U0001F600-\U0001F64F"  # removal of emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+",' ',text)
    
    
    #remove newline char
    text = re.sub('\n', '', text)
    
    return text

The provided code defines a function clean_text() that performs several text cleaning operations using regular expressions (re module). Here's a breakdown of the different steps performed in the function:

Removal of URLs: The regular expression r'https?://\S+|www.\S+|http?://\S+' matches URLs and replaces them with a space.

Removal of HTML tags: The regular expression r'<.*?>' matches HTML tags and replaces them with a space.

Removal of Emoticons, Symbols, and Flags: The regular expression "["u"\U0001F600-\U0001F64F" u"\U0001F300-\U0001F5FF" u"\U0001F680-\U0001F6FF" u"\U0001F1E0-\U0001F1FF" u"\U00002702-\U000027B0" u"\U000024C2-\U0001F251" "]+",' ' removes emoticons, symbols, and flags from the text by replacing them with a space.

Removal of Newline Characters: The '\n' character is replaced with an empty string to remove newline characters from the text.

The function then returns the cleaned text.

In [30]:
books.bookTitle = books.bookTitle.apply(lambda x:clean_text(x))

In [31]:
books

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlM
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...


# Checking Users Data

In [32]:
users.isnull().sum()

userId           0
Location         0
Age         110762
dtype: int64

In [33]:
users.duplicated().sum()

0

In [34]:
users.duplicated('userId').sum()

0

In [35]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   userId    278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [36]:
users['Age'].unique()

array([ nan,  18.,  17.,  61.,  26.,  14.,  25.,  19.,  46.,  55.,  32.,
        24.,  20.,  34.,  23.,  51.,  31.,  21.,  44.,  30.,  57.,  43.,
        37.,  41.,  54.,  42.,  50.,  39.,  53.,  47.,  36.,  28.,  35.,
        13.,  58.,  49.,  38.,  45.,  62.,  63.,  27.,  33.,  29.,  66.,
        40.,  15.,  60.,   0.,  79.,  22.,  16.,  65.,  59.,  48.,  72.,
        56.,  67.,   1.,  80.,  52.,  69.,  71.,  73.,  78.,   9.,  64.,
       103., 104.,  12.,  74.,  75., 231.,   3.,  76.,  83.,  68., 119.,
        11.,  77.,   2.,  70.,  93.,   8.,   7.,   4.,  81., 114., 230.,
       239.,  10.,   5., 148., 151.,   6., 101., 201.,  96.,  84.,  82.,
        90., 123., 244., 133.,  91., 128.,  94.,  85., 141., 110.,  97.,
       219.,  86., 124.,  92., 175., 172., 209., 212., 237.,  87., 162.,
       100., 156., 136.,  95.,  89., 106.,  99., 108., 210.,  88., 199.,
       147., 168., 132., 159., 186., 152., 102., 116., 200., 115., 226.,
       137., 207., 229., 138., 109., 105., 228., 18

- Here we can see we have age more than 100, which is invalid.
- So we take the age from range (5,100) and in place of the nan values we put the mean values

In [37]:
users.loc[(users['Age'] > 100) | (users['Age'] < 5)] = np.nan
users['Age'].fillna((users['Age'].mean()), inplace=True)
users['Age']= users['Age'].astype('int64')
users['Age'].unique()

array([ 34,  18,  17,  61,  26,  14,  25,  19,  46,  55,  32,  24,  20,
        23,  51,  31,  21,  44,  30,  57,  43,  37,  41,  54,  42,  50,
        39,  53,  47,  36,  28,  35,  13,  58,  49,  38,  45,  62,  63,
        27,  33,  29,  66,  40,  15,  60,  79,  22,  16,  65,  59,  48,
        72,  56,  67,  80,  52,  69,  71,  73,  78,   9,  64,  12,  74,
        75,  76,  83,  68,  11,  77,  70,  93,   8,   7,  81,  10,   5,
         6,  96,  84,  82,  90,  91,  94,  85,  97,  86,  92,  87, 100,
        95,  89,  99,  88,  98], dtype=int64)

In [38]:
# For visualization which are the age groups are more 
def age_group(age):
    if age<15:
        x='Children'
    elif age>=15 and age<35:
        x='Youth'
    elif age>=35 and age<65:
        x='Adults'
    else:
        x='Senior Citizens'
    return x

- Here we have categories the data into different age groups for visualization purpose.

In [39]:
users['Age_group']=users['Age'].apply(lambda x: age_group(x))

#### In below cells we are extracting the Country name from the location column,

In [40]:
# defining a function to extract the country names
def get_country(x):
    if isinstance(x, float):
        x = str(x)
    return x.split(',')[-1]

In [41]:
#applying the function the location column
new_user = users.copy(deep=True)
new_user['country'] = users['Location'].apply(get_country)

In [42]:
new_user

,userId,Location,Age,Age_group,country
0,1.0,"nyc, new york, usa",34,Youth,usa
1,2.0,"stockton, california, usa",18,Youth,usa
2,3.0,"moscow, yukon territory, russia",34,Youth,russia
3,4.0,"porto, v.n.gaia, portugal",17,Youth,portugal
4,5.0,"farnborough, hants, united kingdom",34,Youth,united kingdom
...,...,...,...,...,...
278853,278854.0,"portland, oregon, usa",34,Youth,usa
278854,278855.0,"tacoma, washington, united kingdom",50,Adults,united kingdom
278855,278856.0,"brampton, ontario, canada",34,Youth,canada
278856,278857.0,"knoxville, tennessee, usa",34,Youth,usa


In [43]:
new_user['country']

0                     usa
1                     usa
2                  russia
3                portugal
4          united kingdom
               ...       
278853                usa
278854     united kingdom
278855             canada
278856                usa
278857            ireland
Name: country, Length: 278858, dtype: object

In [44]:
# TO know the unique values in the country column
new_user['country'].unique()

array([' usa', ' russia', ' portugal', ' united kingdom', ' canada',
       ' spain', ' australia', ' brazil', '', ' germany', ' mexico',
       ' china', ' italy', ' distrito federal', ' france', ' netherlands',
       ' iraq', ' finland', ' new zealand', ' india', ' ghana',
       ' switzerland', ' slovenia', ' iran', ' peru',
       ' bosnia and herzegovina', ' vietnam', ' sri lanka', ' pakistan',
       ' denmark', ' belgium', ' malaysia', ' u.a.e', ' turkey',
       ' philippines', ' greece', 'nan', ' norway', ' kuwait', ' chile',
       ' quit', ' lj', ' taiwan', ' denmark"', ' españa', ' sweden',
       ' argentina', ' nigeria', ' london', ' bulgaria', ' austria',
       ' romania', ' singapore', ' egypt', ' tunisia', ' uzbekistan',
       ' qatar', ' syria', ' indonesia', ' sudan', ' saudi arabia',
       ' somewherein space', ' poland', ' thailand', ' ireland',
       ' malaysia"', ' venezuela', ' paraguay', ' mozambique', ' morocco',
       ' afghanistan', ' estonia', ' colom

In [45]:
new_user['country']=new_user['country'].astype('str')

In [46]:
new_user['country'].replace(['', '01776', '02458', '19104', '23232', '30064', '85021', '87510', 'None'],
                        ['others','usa','usa','usa','usa','usa','usa','usa','others'],inplace=True)

# Checking Ratings Data

In [47]:
ratings.isnull().sum()

userId        0
ISBN          0
bookRating    0
dtype: int64

In [48]:
ratings.duplicated().sum()

0

In [49]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   userId      1149780 non-null  int64 
 1   ISBN        1149780 non-null  object
 2   bookRating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [50]:
ratings.bookRating.unique()

array([ 0,  5,  3,  6,  8,  7, 10,  9,  4,  1,  2], dtype=int64)

In [51]:
# To know the top rating
ratings.bookRating.value_counts()

0     716109
8     103736
10     78610
7      76457
9      67541
5      50974
6      36924
4       8904
3       5996
2       2759
1       1770
Name: bookRating, dtype: int64

In [52]:
exp_rating= ratings[ratings['bookRating'] != 0]
imp_rating= ratings[ratings['bookRating'] == 0]

# Merging the Dataset

In [53]:
merged_data = pd.merge(books, ratings, on='ISBN')
merged_data.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlM,userId,bookRating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,2,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,8,5
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,11400,0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,11676,8
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,41385,0


In [54]:
data = pd.merge(merged_data, new_user,  on='userId')
data.head()

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlM,userId,bookRating,Location,Age,Age_group,country
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,2,0,"stockton, california, usa",18,Youth,usa
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,8,5,"timmins, ontario, canada",34,Youth,canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,8,0,"timmins, ontario, canada",34,Youth,canada
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,8,0,"timmins, ontario, canada",34,Youth,canada
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,8,0,"timmins, ontario, canada",34,Youth,canada


# Collaborative Filtering Based Recommender System

- This implementation focuses on finding similar items (books) based on user ratings and recommending books that are similar to a given book.

In [55]:
num_rating_df = data.groupby('bookTitle').count()['bookRating'].reset_index()
num_rating_df.rename(columns={'bookRating':'num_ratings'},inplace=True)
num_rating_df

,bookTitle,num_ratings
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
240503,Ã?Â?lpiraten.,2
240504,Ã?Â?rger mit Produkt X. Roman.,4
240505,Ã?Â?sterlich leben.,1
240506,Ã?Â?stlich der Berge.,3


In [56]:
avg_rating_df = data.groupby('bookTitle').mean()['bookRating'].reset_index()
avg_rating_df.rename(columns={'bookRating':'avg_rating'},inplace=True)
avg_rating_df

,bookTitle,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
240503,Ã?Â?lpiraten.,0.000000
240504,Ã?Â?rger mit Produkt X. Roman.,5.250000
240505,Ã?Â?sterlich leben.,7.000000
240506,Ã?Â?stlich der Berge.,2.666667


In [57]:
popular_df = num_rating_df.merge(avg_rating_df,on='bookTitle')
popular_df

,bookTitle,num_ratings,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
240503,Ã?Â?lpiraten.,2,0.000000
240504,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
240505,Ã?Â?sterlich leben.,1,7.000000
240506,Ã?Â?stlich der Berge.,3,2.666667


In [58]:
x = data.groupby('userId').count()['bookRating'] > 200
regular_reader = x[x].index

Here we are using only that users who had rated more than 200 books to over come the sparsity issue.

In [59]:
regular_reader

Int64Index([   254,   2276,   2766,   2977,   3363,   4017,   4385,   6251,
              6323,   6543,
            ...
            271705, 273979, 274004, 274061, 274301, 274308, 275970, 277427,
            277639, 278418],
           dtype='int64', name='userId', length=807)

In [60]:
filtered_rating = data[data['userId'].isin(regular_reader)]

here we are getting only regular reader from the main data set

In [61]:
filtered_rating

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlM,userId,bookRating,Location,Age,Age_group,country
80,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,11676,8,"n/a, n/a, n/a",34,Youth,n/a
81,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,11676,9,"n/a, n/a, n/a",34,Youth,n/a
82,0671870432,PLEADING GUILTY,Scott Turow,1993,Audioworks,http://images.amazon.com/images/P/0671870432.0...,11676,8,"n/a, n/a, n/a",34,Youth,n/a
83,0771074670,Nights Below Station Street,David Adams Richards,1988,Emblem Editions,http://images.amazon.com/images/P/0771074670.0...,11676,0,"n/a, n/a, n/a",34,Youth,n/a
84,0440234743,The Testament,John Grisham,1999,Dell,http://images.amazon.com/images/P/0440234743.0...,11676,9,"n/a, n/a, n/a",34,Youth,n/a
...,...,...,...,...,...,...,...,...,...,...,...,...
893253,0451406184,Petticoats and Pistols (Topaz Historical Roman...,Margaret Brownley,1995,Topaz,http://images.amazon.com/images/P/0451406184.0...,72352,0,"samieira, pontevedra, spain",49,Adults,spain
893254,0505521423,Savage Whispers (Savage),Cassie Edwards,1996,Love Spell,http://images.amazon.com/images/P/0505521423.0...,72352,0,"samieira, pontevedra, spain",49,Adults,spain
893255,0451408438,Silver Wing (Topaz Historical Romance),Cassie Edwards,1999,Topaz,http://images.amazon.com/images/P/0451408438.0...,72352,0,"samieira, pontevedra, spain",49,Adults,spain
893256,0821764667,Her Only Hero (Zebra Splendor Historical Roman...,Jo Ann Ferguson,2000,Kensington Publishing Corporation,http://images.amazon.com/images/P/0821764667.0...,72352,8,"samieira, pontevedra, spain",49,Adults,spain


In [62]:
# here we are getting only those books which are rated more than 50 times
y = filtered_rating.groupby('bookTitle').count()['bookRating']>=50
famous_books = y[y].index
famous_books

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Day Late and a Dollar Short', 'A Fine Balance',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='bookTitle', length=699)

In [63]:
# here we have the final data with readers who have rated more than 200 books and books with more than 50 ratings
final_ratings = filtered_rating[filtered_rating['bookTitle'].isin(famous_books)]
final_ratings

,ISBN,bookTitle,bookAuthor,yearOfPublication,publisher,imageUrlM,userId,bookRating,Location,Age,Age_group,country
81,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,11676,9,"n/a, n/a, n/a",34,Youth,n/a
84,0440234743,The Testament,John Grisham,1999,Dell,http://images.amazon.com/images/P/0440234743.0...,11676,9,"n/a, n/a, n/a",34,Youth,n/a
85,0452264464,Beloved (Plume Contemporary Fiction),Toni Morrison,1994,Plume,http://images.amazon.com/images/P/0452264464.0...,11676,8,"n/a, n/a, n/a",34,Youth,n/a
88,0971880107,Wild Animus,Rich Shapero,2004,Too Far,http://images.amazon.com/images/P/0971880107.0...,11676,6,"n/a, n/a, n/a",34,Youth,n/a
89,0345402871,Airframe,Michael Crichton,1997,Ballantine Books,http://images.amazon.com/images/P/0345402871.0...,11676,0,"n/a, n/a, n/a",34,Youth,n/a
...,...,...,...,...,...,...,...,...,...,...,...,...
819056,0670894184,Back Roads,Tawni O'Dell,2000,Viking Books,http://images.amazon.com/images/P/0670894184.0...,165232,5,"saint albans, west virginia, usa",23,Youth,usa
819061,0670858692,Rose Madder,Stephen King,1995,Viking Books,http://images.amazon.com/images/P/0670858692.0...,165232,0,"saint albans, west virginia, usa",23,Youth,usa
893069,1551668874,Heartbreaker,Linda Howard,2002,Mira,http://images.amazon.com/images/P/1551668874.0...,72352,0,"samieira, pontevedra, spain",49,Adults,spain
893106,0671670069,Guardian Angel,Julie Garwood,1990,Pocket,http://images.amazon.com/images/P/0671670069.0...,72352,10,"samieira, pontevedra, spain",49,Adults,spain


In [64]:
final_ratings['userId'].unique()

array([ 11676,  85526,  96054, 177458, 110912, 137688, 197659,  36836,
        46398, 113270, 113519, 200674, 211919, 230522, 238557, 245827,
        52614, 166596,   3363, 174304, 177090,   2977,   7346,  13552,
        14521,  16795,  23768,  26544,  26593,  30487,  30533,  31315,
        31556,  35857,  35859,  36606,  39646,  40889,  52584,  55492,
        69232,  76352,  76942,  78834,  78973,  79186,  82893,  84024,
        91832,  93363, 101209, 102647, 102967, 105979, 107784, 110973,
       112001, 115003, 115120, 123790, 123981, 124363, 124487, 125039,
       125519, 126492, 127200, 131046, 131837, 135149, 135831, 137589,
       142093, 143253, 144531, 145619, 146113, 147965, 153718, 155495,
       160819, 164323, 167800, 170518, 172742, 173415, 173684, 185384,
       186570, 195694, 197364, 198711, 203968, 207782, 208147, 214786,
       216683, 217375, 221445, 224435, 225810, 225989, 229011, 231237,
       232131, 232945, 234623, 235842, 235935, 236172, 240144, 240567,
      

In [65]:
pt = final_ratings.pivot_table(index='bookTitle',columns='userId',values='bookRating')

In [66]:
pt.fillna(0,inplace=True)

In [67]:
pt

userId,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
bookTitle,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
from sklearn.metrics.pairwise import cosine_similarity

In [69]:
#here we are finding cosine similarity of every rows with each other. 
#Basically we are finding euclidean distance of each book with other book.
similarity_scores = cosine_similarity(pt)

In [70]:
similarity_scores

array([[1.        , 0.10377079, 0.01220856, ..., 0.12110367, 0.07347567,
        0.04316046],
       [0.10377079, 1.        , 0.23927158, ..., 0.07534752, 0.16973515,
        0.14433158],
       [0.01220856, 0.23927158, 1.        , ..., 0.04558758, 0.04938579,
        0.10796119],
       ...,
       [0.12110367, 0.07534752, 0.04558758, ..., 1.        , 0.07085128,
        0.0196177 ],
       [0.07347567, 0.16973515, 0.04938579, ..., 0.07085128, 1.        ,
        0.10602962],
       [0.04316046, 0.14433158, 0.10796119, ..., 0.0196177 , 0.10602962,
        1.        ]])

In [71]:
similarity_scores.shape

(699, 699)

In [72]:
def recommend(book_name):
    # index fetch
    index = np.where(pt.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[0:6]
    
    data = []
    for i in similar_items:
        item = []
        temp_df = books[books['bookTitle'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('bookTitle')['bookTitle'].values))
        item.extend(list(temp_df.drop_duplicates('bookTitle')['bookAuthor'].values))
        item.extend(list(temp_df.drop_duplicates('bookTitle')['imageUrlM'].values))
        item.extend(list(temp_df.drop_duplicates('bookTitle')['ISBN'].values))
        data.append(item)
    
    return data

In [73]:
def topBook(userID):
    return recommend(final_ratings.loc[final_ratings['userId'] == userID]['bookTitle'].values[0])

In [74]:
topBook(78783)

[['Airframe',
  'Michael Crichton',
  'http://images.amazon.com/images/P/0345402871.01.MZZZZZZZ.jpg',
  '0345402871'],
 ['Sphere',
  'MICHAEL CRICHTON',
  'http://images.amazon.com/images/P/0345353145.01.MZZZZZZZ.jpg',
  '0345353145'],
 ['Absolute Power',
  'David Baldacci',
  'http://images.amazon.com/images/P/0446519960.01.MZZZZZZZ.jpg',
  '0446519960'],
 ['Welcome to the World, Baby Girl!',
  'Fannie Flagg',
  'http://images.amazon.com/images/P/080411868X.01.MZZZZZZZ.jpg',
  '080411868X'],
 ['Jurassic Park',
  'Michael Crichton',
  'http://images.amazon.com/images/P/0345370775.01.MZZZZZZZ.jpg',
  '0345370775'],
 ['Fatal Cure',
  'Robin Cook',
  'http://images.amazon.com/images/P/0425145638.01.MZZZZZZZ.jpg',
  '0425145638']]

# Hybrid Model

In [75]:
# Content-based filtering
book_features = data[['bookTitle', 'bookAuthor', 'imageUrlM']]  
book_features = book_features.drop_duplicates('bookTitle').reset_index(drop=True)

import pickle
pickle_data = [data, num_rating_df, avg_rating_df, popular_df, filtered_rating, famous_books, final_ratings, pt, similarity_scores, book_features]

with open('updated_recommendation_pickle.pkl', 'wb') as file:
    pickle.dump(pickle_data, file)

data---------------------------0
num_rating_df------------------1
avg_rating_df------------------2
popular_df---------------------3
filtered_rating---------------4
famous_books,-----------------5
final_ratings,-----------------6
pt,----------------------------7
similarity_scores,-------------8
book_features,------------------9
hybrid_recommend, -------------10
recommend_for_user-------------11

In [3]:
import pickle
with open('updated_recommendation_pickle.pkl', 'rb') as file:
    pickle_data = pickle.load(file)

In [4]:
def hybrid_recommend(book_name):
    # Content-based filtering
    book_features_subset = pickle_data[9][pickle_data[9]['bookTitle'] == book_name]
    if book_features_subset.empty:
        return []  # Return empty lis if book not found in book features
    book_subset_indices = book_features_subset.index.tolist()
    book_subset_similarity_scores = pickle_data[8][:, book_subset_indices].flatten()
    content_based_scores = list(enumerate(book_subset_similarity_scores))

    # Collaborative filtering
    collaborative_scores = list(enumerate(pickle_data[8][pickle_data[7].index == book_name].flatten()))

    # Combine scores from both approaches
    combined_scores = [(idx, content_based_scores[idx][1] + collaborative_scores[idx][1]) for idx in range(len(content_based_scores))]

    # Sort the combined scores
    combined_scores = sorted(combined_scores, key=lambda x: x[1], reverse=True)[:6]

    # Retrieve recommended books
    recommended_books = []
    for i in combined_scores:
        temp_df = pickle_data[0][pickle_data[0]['bookTitle'] == pickle_data[7].index[i[0]]]
        recommended_books.append((temp_df['bookTitle'].values[0], temp_df['bookAuthor'].values[0], temp_df['imageUrlM'].values[0]))
    
    return recommended_books

In [13]:
def recommend_for_user(userID, location, age):
    l2 = []
    if (userID in pickle_data[6]['userId'].unique()) & (userID in pickle_data[0]['userId'].unique()) :
        # User exists in the dataset, recommend based on their ratings
        user_books = pickle_data[6].loc[pickle_data[6]['userId'] == userID]['bookTitle'].values[:3]
        for book in user_books:
            recommendations = hybrid_recommend(book)
            l2.extend(recommendations[:3])  # Take the top 2 recommendations for each book
        return l2
    
    elif ((userID not in pickle_data[6]['userId'].unique()) & (userID in pickle_data[0]['userId'].unique())):
        most_rated_book = pickle_data[6].groupby('bookTitle').count().sort_values('bookRating', ascending=False).index[0:10]
        l1 = list(most_rated_book)
        for i in l1:
            l2.append((i, pickle_data[6].loc[pickle_data[6]['bookTitle'] == i]['bookAuthor'].values[0], pickle_data[6].loc[pickle_data[6]['bookTitle'] == i]['imageUrlM'].values[0]))
        return l2
        
    elif ((userID not in pickle_data[6]['userId'].unique()) & (userID not in pickle_data[0]['userId'].unique()) & (location!='none') & (age==0)):
        location_wise = pickle_data[0].groupby(['bookTitle', 'country']).size().reset_index(name='rating_count').sort_values('rating_count', ascending=False)
        books_for_country = location_wise[location_wise['country'] == " "+location].head(10)
        books_for_country = books_for_country['bookTitle']
        return(books_for_country)
    
    elif ((userID not in pickle_data[6]['userId'].unique()) & (userID not in pickle_data[0]['userId'].unique()) & (location== 'none') & (age!=0)):
        age_wise = pickle_data[0].groupby(['bookTitle', 'Age_group']).size().reset_index(name='rating_count').sort_values('rating_count', ascending=False)
        books_by_age = age_wise[age_wise['Age_group'] == age].head(10)
        books_by_age = books_by_age['bookTitle']
        return(books_by_age)
        
    elif ((userID not in pickle_data[6]['userId'].unique()) & (userID not in pickle_data[0]['userId'].unique()) & (location== 'none') & (age==0)):
        most_rated_book = pickle_data[6].groupby('bookTitle').count().sort_values('bookRating', ascending=False).index[0:10]
        l1 = list(most_rated_book)
        for i in l1:
            l2.append((i, pickle_data[6].loc[pickle_data[6]['bookTitle'] == i]['bookAuthor'].values[0], pickle_data[6].loc[pickle_data[6]['bookTitle'] == i]['imageUrlM'].values[0]))
        return l2

    elif ((userID not in pickle_data[6]['userId'].unique()) & (userID not in pickle_data[0]['userId'].unique()) & (location!= 'none') & (age!=0)):
        books_by_country_n_age = pickle_data[0].groupby(['bookTitle', 'Age_group', 'country']).size().reset_index(name='rating_count').sort_values('rating_count', ascending=False)
        books_by_country_n_age = books_by_country_n_age[(books_by_country_n_age['country'] == " "+location) & (books_by_country_n_age['Age_group'] == age)].head(10)
        books_by_country_n_age = books_by_country_n_age['bookTitle']
        l8 = list(books_by_country_n_age)
        for i in l8:
            l2.append((i, pickle_data[6].loc[pickle_data[6]['bookTitle'] == i]['bookAuthor'].values[0], pickle_data[6].loc[pickle_data[6]['bookTitle'] == i]['imageUrlM'].values[0]))
        return l2
    
    else:
        print('xyz')
